# Contamination check with TPFPLOTTER

```bash
git clone https://github.com/jlillo/tpfplotter.git
```


A **contaminant** is a star, different from your target, which light is entering in your photometric aperture (i.e. the mask), causing a dilution of the transit. 

$\Delta m$ is the magnitude difference between the contaminant and your target, in the Gaia G band 

Use the TFP plot to identify possible contaminants entering in your default mask. 
- If the contaminant is within the mask, but it $\Delta m > 4$ (the contaminant is fainter than the target of 4 magnitude in G band), you can ignore it 
- if $2 < \Delta m < 4$ (with the contaminant _fainter_ than your target), but the contaminant is at leat 2 pixels away, for now you can ignore the contaminant. Later on you will need to compare the transit depth with the value in literature and with the TASTE light curve, check if they are consistent, and decide if you want to repeat the light curve extraction including the contaminant
- if $\Delta m < 2$ you should include the contaminant in the mask

To retrieve the _Gaia ID_ of the contaminat, run again tpfplotter with the additional flag `--SAVEGAIA`

```bash
  python tpfplotter.py xxxxxxxxx --maglim 6 --SAVEGAIA
```

A file with the name `Gaia_TICxxxxxxxxx_Syy.dat`, with the proper TIC number and sector of observations, will be written in the same folder of the pdf 
The file will be similar to this one:
```text

# ID GaiaID x y Dist_pix Dist_arcsec Gmag InAper
1 5284517766615492736 406.49884892803937 1809.0153210210442 0.0 0.0 12.0665 1
2 5284517762316055552 407.07494874794327 1808.5037379540615 0.7704597568443792 16.179654893731964 17.5731 1
3 5284519037925814912 406.8111820069747 1806.1456608843614 2.886607221681461 60.61875165531068 15.5482 0
```
The first row is your targetm, the other object are listed with the same ID number as in the figure. 
To find the name of the target, go to Simbad -> Query by identifier 
http://simbad.u-strasbg.fr/simbad/sim-fid
and enter in the search box:
`Gaia DR2 xxxxxxxxxxxxxxxxxxx`
where xxxxxxxxxxxxxxxxxxx is the GaiaID of the target you are interested in searching. 
You will either get the Simbad webpage of the target, or an error like the one below, with the link to the actual Gaia DR2 entry in the Vizier catalogue:
`
Identifier not found in the database : Gaia DR2 5284517762316055552
Recognized name catalogue in VizieR : Gaia DR2 5284517762316055552
`
In any case, you just need to copy the RA and DEC of the target and put them in the `Search the TESS Candidate Target List` box of ExoFOP, reachable from the main web page or directly from this link:
https://exofop.ipac.caltech.edu/tess/search.php

Now that we have the TIC number of the contaminant from the ExoFOP page, you may want to know the TESS default aperture for the contaminant and avoid the daunting task or choosing the mask by yourself. 
If you have a single sector, you can simply:
- download the TPD plot of the contaminant using tpfplotter
- update the mask of your target by adding the pixels of the contaminant 
You have multiple sectors:
- you can still use tpfplotter with the `--sector nn` flag
```bash
python tpfplotter.py xxxxxxxxx --maglim 6 --sector nn 
```
OR
- Visualize the standard pipeline mask on top of the image by running search_targetpixelfile and repeating the same steps as done for the target star 
but **remember** each sector will require a differen mask!